# **Import libraries**

In [1]:
import csv

# **Personalized functions** 

In [2]:
def hasNumber(stringVal) :
  return any(elem.isdigit() for elem in stringVal)

In [3]:
def getDigit(stringVal) :
  onlyDigit = ''
  for i in stringVal :
    if (i.isdigit()) :
      onlyDigit += i
  return onlyDigit

In [4]:
def changeToNUM(measurement_list) :
  new_measurement_list = []
  for j in measurement_list : #measure
    if (hasNumber(j) == False) : #숫자가 하나도 없고 전부 문자열인 경우
      new_measurement_list.append(0)
    elif (j == '') : #빈 값인 경우 읽기 때문에 0으로 변경해줌
      new_measurement_list.append(0)

    else : #'1 1/2 oz' 형식으로 되어있음 따라서 공백으로 자르고 처리
      slice_str = []
      slice_str = j.split()
      total_val = 0
        
      for n in slice_str : #잘라진 '1' '1/2' 'oz' 처리
        if (n.isalpha()) :
          pass
        else :
          if (n.find('/') == -1) : #그냥 숫자인 경우
            num_tmp = getDigit(n)
            if (num_tmp == '') :
              num_tmp = 0
            total_val += int(num_tmp)
          else : # 1/2 인 경우
            idx = n.find('/')
            divide_val = int(n[:idx]) / int(n[idx+1:])
            total_val += divide_val
      new_measurement_list.append(total_val)
  return new_measurement_list

# **Read data from csv and pre-process data**

# **Label and ingredient set**

In [5]:
label = []
ingredient = []

f = open('mr-boston-flattened.csv','r')
rdr = csv.reader(f)

first_line_pass = 0
line_count = 0
for line in rdr:
  if (first_line_pass == 0) :
    first_line_pass += 1
    pass
  else :
    line_count += 1
    label.append(line[1])
    
    for i in range(3,15,2) :
      ingredient.append(line[i])

f.close()

label = set(label)
label = list(label)

#add wrong label
label.append("Wrong")

ingredient = set(ingredient)
ingredient = list(ingredient)

print("line len ", line_count)
print("label len", len(label))
print("label info :",label)
print("ingredient len", len(ingredient))

line len  990
label len 12
label info : ['Rum - Daiquiris', 'Tequila', 'Cocktail Classics', 'Non-alcoholic Drinks', 'Whiskies', 'Shooters', 'Rum', 'Brandy', 'Cordials and Liqueurs', 'Gin', 'Vodka', 'Wrong']
ingredient len 688


In [6]:
label = ['' for i in range(line_count)]
ing = [[0 for col in range(len(ingredient))] for row in range(line_count)]

In [7]:
# my_data = []
# f = open('mr-boston-flattened.csv','r')
# rdr = csv.reader(f)

# line_count = 0
# for line in rdr :
#   if (line_count == 0) :
#     line_count += 1
#     pass
#   else :
#     #변수 초기화
#     cocktail = [] #우리 데이터 한 row
#     cocktail_ingredient_measure = [] #칵테일 성분 정보(딕셔너리의 리스트로 구성 [{성분1 : 용량}, {성분2 : 용량}, ... ])
#     measurement_list = [] 
#     ingred_idx = 3 #용량에 mapping 되는 ingredient idx
#     new_measurement_list = [] #변경한 값 담는 배열
    
#     for i in range(2,14,2) : #각 row에 measurement 뽑아서 list에 넣기
#       measurement_list.append(line[i])

#     for j in measurement_list : #measure
#       if (hasNumber(j) == False) : #숫자가 하나도 없고 전부 문자열인 경우
#         new_measurement_list.append(0)
#       elif (j == '') : #빈 값인 경우 읽기 때문에 0으로 변경해줌
#         new_measurement_list.append(0)

#       else : #'1 1/2 oz' 형식으로 되어있음 따라서 공백으로 자르고 처리
#         slice_str = []
#         slice_str = j.split()
#         total_val = 0
        
#         for n in slice_str : #잘라진 '1' '1/2' 'oz' 처리
#           if (n.isalpha()) :
#             pass
#           else :
#             if (n.find('/') == -1) : #그냥 숫자인 경우
#               num_tmp = getDigit(n)

#               if (num_tmp == '') :
#                 num_tmp = 0

#               total_val += int(num_tmp)
#             else : # 1/2 인 경우
#               idx = n.find('/')
#               divide_val = int(n[:idx]) / int(n[idx+1:])
#               total_val += divide_val
#         new_measurement_list.append(total_val)
        

# ### 여기까지 오면 new_measurement_list에는 숫자만 들어있음
#     total_measurement_value = sum(new_measurement_list)

#     for j in new_measurement_list :
#       if (line[ingred_idx] != '') :

#         ing_measure_pair = {line[ingred_idx] : j}
#         cocktail_ingredient_measure.append(ing_measure_pair)
      
#       ingred_idx += 2

#     #칵테일 이름
#     cocktail.append(line[0])
#     #칵테일 라벨
#     cocktail.append(line[1])
#     #성분 전체 용량
#     cocktail.append(total_measurement_value)
#     #각 성분별 성분명, 용량 딕셔너리
#     cocktail.append(cocktail_ingredient_measure)
#     my_data.append(cocktail)

# f.close()

In [11]:
f = open('mr-boston-flattened.csv','r')
rdr = csv.reader(f)

first_line_pass = 0
line_count = 0
for line in rdr :
  if (first_line_pass == 0) :
    first_line_pass += 1
    pass
  else :

    #변수 초기화
    measurement_list = [] 
    ingred_idx = 3 #용량에 mapping 되는 ingredient idx
    
    for i in range(2,14,2) : #각 row에 measurement 뽑아서 list에 넣기
      measurement_list.append(line[i])

    #ingredient string 을 숫자만 남김
    new_measurement_list = changeToNUM(measurement_list)
    
    #전체 용량 계산하는 부분 필요한 경우 다시 활성화
    #total_measurement_value = sum(new_measurement_list)

    #칵테일 라벨
    label[line_count] = line[1]

    #숫자로 변환한 것들에 대응하는 재료를 전체 ingredient 리스트에서 해당 하는 인덱스 찾은 후 위치에 값 설정하기
    for j in new_measurement_list :
      if (line[ingred_idx] != '') :
        ing_measure_pair = {line[ingred_idx] : j}
        idx = ingredient.index(line[ingred_idx])
        ing[line_count][idx] = j
      ingred_idx += 2

    line_count += 1 #line idx 역할

f.close()

# **See the ingredient list**

In [12]:
for i in ing :
  print(i)

print("ing row num",len(ing))
print("ing col num",len(ing[0]))

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

# **See the label list**

In [10]:
for i in label :
  print(i)

print("label row num",len(label))

Cocktail Classics
Cocktail Classics
Cordials and Liqueurs
Cocktail Classics
Cocktail Classics
Whiskies
Cocktail Classics
Cocktail Classics
Cocktail Classics
Cordials and Liqueurs
Whiskies
Cocktail Classics
Cocktail Classics
Cocktail Classics
Cocktail Classics
Cocktail Classics
Cordials and Liqueurs
Cocktail Classics
Cocktail Classics
Cocktail Classics
Whiskies
Cocktail Classics
Cocktail Classics
Cocktail Classics
Cordials and Liqueurs
Cocktail Classics
Cocktail Classics
Cocktail Classics
Cocktail Classics
Cocktail Classics
Whiskies
Cocktail Classics
Cocktail Classics
Cocktail Classics
Cocktail Classics
Cocktail Classics
Cordials and Liqueurs
Cocktail Classics
Whiskies
Brandy
Cocktail Classics
Cocktail Classics
Cocktail Classics
Cordials and Liqueurs
Cocktail Classics
Cocktail Classics
Whiskies
Cordials and Liqueurs
Cocktail Classics
Cocktail Classics
Cocktail Classics
Whiskies
Cocktail Classics
Cocktail Classics
Cocktail Classics
Cocktail Classics
Whiskies
Cocktail Classics
Cocktail Cl